In [2]:
# !wget https://download.pytorch.org/tutorial/data.zip
# !unzip data

!git clone https://github.com/NoCodeProgram/deepLearning.git

fatal: destination path 'deepLearning' already exists and is not an empty directory.


In [3]:
# import torch
# import torchvision
# import matplotlib.pyplot as plt

# if torch.backends.mps.is_available():
#     my_device = torch.device('mps')
# elif torch.cuda.is_available():
#     my_device = torch.device('cuda')
# else:
#     my_device = torch.device('cpu')


In [4]:
import pandas as pd
import torch

df = pd.read_csv('./deepLearning/rnn/name_country.csv')
text_data = df['Name'].tolist()
label_data = df['Country'].tolist()

country_list = sorted(set(label_data))
country_count = len(country_list)

data_dict = {} #key-country, value - list of names
for name, country in zip(text_data, label_data):
    if country not in data_dict:
        data_dict[country] = []
    data_dict[country].append(name)

print(data_dict)


{'Czech': ['abl', 'adsit', 'ajdrna', 'alt', 'antonowitsch', 'antonowitz', 'bacon', 'ballalatak', 'ballaltick', 'bartonova', 'bastl', 'baroch', 'benesch', 'betlach', 'biganska', 'bilek', 'blahut', 'blazek', 'blazek', 'blazejovsky', 'blecha', 'bleskan', 'blober', 'bock', 'bohac', 'bohunovsky', 'bolcar', 'borovka', 'borovski', 'borowski', 'borovsky', 'brabbery', 'brezovjak', 'brousil', 'bruckner', 'buchta', 'cablikova', 'camfrlova', 'cap', 'cerda', 'cermak', 'chermak', 'cermak', 'cernochova', 'cernohous', 'cerny', 'cerney', 'cerny', 'cerv', 'cervenka', 'chalupka', 'charlott', 'chemlik', 'chicken', 'chilar', 'chromy', 'cihak', 'clineburg', 'klineberg', 'cober', 'colling', 'cvacek', 'czabal', 'damell', 'demall', 'dehmel', 'dana', 'dejmal', 'dempko', 'demko', 'dinko', 'divoky', 'dolejsi', 'dolezal', 'doljs', 'dopita', 'drassal', 'driml', 'duyava', 'dvorak', 'dziadik', 'egr', 'entler', 'faltysek', 'faltejsek', 'fencl', 'fenyo', 'fillipova', 'finfera', 'finferovy', 'finke', 'fojtikova', 'fremu

In [5]:
all_letters = "abcdefghijklmnopqrstuvwxyz ,'"
n_letters = len(all_letters)

def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    for char_idx, char in enumerate(name):
        letter_idx = all_letters.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor



In [6]:
import torch.nn as nn
#Try other arch
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn_model = RNN(n_letters, n_hidden, country_count)

In [7]:
import random
from torch.optim import Adam, SGD
loss_fn = nn.NLLLoss()

optimizer = SGD(rnn_model.parameters(), lr=0.001) #Adam 1.2647557258605
iter_count = 10000000
crnt_loss = 0.
for iter_idx in range(iter_count):
    rnn_model.train() #not balanced
    random_country = random.choice(list(data_dict.keys()))
    random_name = random.choice(data_dict[random_country])

    name_tensor = nameToTensor(random_name)
    country_tensor = torch.tensor([country_list.index(random_country)], dtype=torch.long)
    hidden = rnn_model.initHidden()
    rnn_model.zero_grad()

    for char_idx in range(len(random_name)):
        char_tensor = name_tensor[char_idx]
        output, hidden = rnn_model(char_tensor[None,:], hidden)
    
    loss = loss_fn(output, country_tensor)
    crnt_loss += loss
    loss.backward()
    optimizer.step()

    # for p in rnn_model.parameters():
    #     p.data.add_(p.grad.data, alpha=-0.0001)
    if iter_idx % 5000 == 0 and iter_idx != 0:
        print(f'iter idx {iter_idx}, loss : {crnt_loss/5000}')
        crnt_loss = 0.
        # print(f'batch idx {train_idx}, loss : {loss.item()}')



iter idx 5000, loss : 2.8702499866485596
iter idx 10000, loss : 2.815654993057251
iter idx 15000, loss : 2.7457234859466553
iter idx 20000, loss : 2.580186128616333
iter idx 25000, loss : 2.459829330444336
iter idx 30000, loss : 2.3688745498657227
iter idx 35000, loss : 2.313839912414551
iter idx 40000, loss : 2.255200147628784
iter idx 45000, loss : 2.177395820617676
iter idx 50000, loss : 2.108016014099121
iter idx 55000, loss : 2.053006172180176
iter idx 60000, loss : 2.013638496398926
iter idx 65000, loss : 1.9488525390625
iter idx 70000, loss : 1.9260715246200562
iter idx 75000, loss : 1.8693021535873413
iter idx 80000, loss : 1.83756422996521
iter idx 85000, loss : 1.8093420267105103
iter idx 90000, loss : 1.7844990491867065
iter idx 95000, loss : 1.7543538808822632
iter idx 100000, loss : 1.714129090309143
iter idx 105000, loss : 1.709670901298523
iter idx 110000, loss : 1.6838581562042236
iter idx 115000, loss : 1.6727879047393799
iter idx 120000, loss : 1.640411138534546
iter 

KeyboardInterrupt: 

In [17]:
test_name = 'jiguem'

rnn_model.eval()
name_tensor = nameToTensor(test_name)
hidden = rnn_model.initHidden()
for char_idx in range(len(test_name)):
    char_tensor = name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:], hidden)
country_idx = torch.argmax(output, dim=1).item()
print(country_list[country_idx])

#data purification is much more important. but skip it
#traning/test set split is important but we ignored it


English


In [ ]:
# we may add additional code for this
import matplotlib.pyplot as plt

# Keep track of correct guesses in a confusion matrix
confusion = torch.zeros(country_count, country_count)
n_confusion = 10000

# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn_model.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn_model(line_tensor[i], hidden)

    return output

# Go through a bunch of examples and record which are correctly guessed
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    guess, guess_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][guess_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([''] + all_categories, rotation=90)
ax.set_yticklabels([''] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

# sphinx_gallery_thumbnail_number = 2
plt.show()

NameError: name 'randomTrainingExample' is not defined